In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [11]:
# Загрузка данных из CSV
df = pd.read_csv('comments.csv')

# Разделение на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [3]:
# Определение класса Dataset для загрузки данных
class ToxicCommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Инициализация токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Создание DataLoader для обучающей и тестовой выборок
train_dataset = ToxicCommentDataset(train_df['comment'], train_df['toxic'], tokenizer, max_length=128)
test_dataset = ToxicCommentDataset(test_df['comment'], test_df['toxic'], tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Инициализация оптимизатора и функции потерь
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Обучение модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Оценка модели на тестовой выборке
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Вывод метрик на каждой эпохе
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds)
    print(f'Epoch {epoch + 1}/{num_epochs}, Accuracy: {accuracy:.4f}\n{report}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Accuracy: 0.8148
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      1944
           1       0.79      0.59      0.67       939

    accuracy                           0.81      2883
   macro avg       0.81      0.76      0.77      2883
weighted avg       0.81      0.81      0.81      2883

Epoch 2/3, Accuracy: 0.8644
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      1944
           1       0.80      0.78      0.79       939

    accuracy                           0.86      2883
   macro avg       0.85      0.84      0.84      2883
weighted avg       0.86      0.86      0.86      2883

Epoch 3/3, Accuracy: 0.8668
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      1944
           1       0.86      0.70      0.77       939

    accuracy                           0.87      2883
   macro avg       0.87      0.82      0.84 

In [4]:
tokenizer.save_pretrained('comment_classifier')
model.save_pretrained('comment_classifier')

# Проверка

In [2]:
tokenizer = BertTokenizer.from_pretrained('comment_classifier')
model = BertForSequenceClassification.from_pretrained('comment_classifier')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [3]:
# Пример текста для проверки
sample_text = "Завтра будет хорошая погода ."

# Токенизация и подготовка текста для модели
encoded_text = tokenizer(sample_text, return_tensors='pt', truncation=True, padding=True, max_length=128)
input_ids = encoded_text['input_ids'].to(device)
attention_mask = encoded_text['attention_mask'].to(device)

# Предсказание
model.eval()
with torch.no_grad():
    output = model(input_ids, attention_mask=attention_mask)

# Получение предсказания
logits = output.logits
predicted_class = torch.argmax(logits, dim=1).item()

# Вывод результата
if predicted_class == 1:
    print("Текст токсичен.")
else:
    print("Текст не токсичен.")

Текст не токсичен.


In [12]:
df["toxic"].value_counts()

toxic
0.0    9586
1.0    4826
Name: count, dtype: int64